# <div align="center"> 目标探测 </div>

In [4]:
%reload_ext autoreload
%autoreload 2

In [7]:
from k12libs.utils.nb_easy import k12ai_post_cv_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_del_data
from k12libs.utils.nb_easy import k12ai_print

## 1 训练

In [17]:
from k12libs.templates.det.voc import SSD_VOC_CONFIG

config = SSD_VOC_CONFIG
config['solver']['max_epoch'] = 2
config['train']['batch_size'] = 8
config['network']['model_name'] = 'vgg16_ssd300'

### train.stop

In [22]:
cv_stop = k12ai_post_cv_request(
    uri='k12ai/framework/train',
    op='train.stop',
    user='test',
    uuid='voc',
)
k12ai_print(cv_stop)

{
    "req": {
        "op": "train.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "voc",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/voc/train.stop"
}


In [10]:
k12ai_del_data(cv_stop['key'])
k12ai_print(k12ai_get_data(cv_stop['key'], 'error', waitcnt=3))

### train.start

In [18]:
cv_start = k12ai_post_cv_request(
    uri='k12ai/framework/train',
    op='train.start',
    user='test',
    uuid='voc',
    params=config
)
k12ai_print(cv_start)

{
    "req": {
        "op": "train.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "voc",
        "service_params": {
            "dataset": "default",
            "task": "det",
            "method": "single_shot_detector",
            "data": {
                "num_classes": 21,
                "data_dir": "/data/datasets/cv/VOC07+12_DET",
                "image_tool": "cv2",
                "input_mode": "BGR",
                "keep_difficult": false,
                "workers": 1,
                "mean_value": [
                    104,
                    117,
                    123
                ],
                "normalize": {
                    "div_value": 1,
                    "mean": [
                        104.0,
                        117.0,
                        123.0
                    ],
                    "std": [
                        1.0,
                        1.0,
                        1.0
                 

In [19]:
k12ai_del_data(cv_start['key'])

'success'

In [21]:
# wait long time, if no result, try again
k12ai_print(k12ai_get_data(cv_start['key'], 'metrics', waitcnt=30))
k12ai_print(k12ai_get_data(cv_start['key'], 'error'))
k12ai_print(k12ai_get_data(cv_start['key'], 'status'))

[
    {
        "key": "framework/test/voc/train.start/metrics/201912200037",
        "value": {
            "version": "0.1.0",
            "type": "metrics",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "voc",
            "timestamp": 1576773475661,
            "datetime": "20191220003755",
            "metrics": {
                "training_epochs": 1,
                "training_loss": 14.74823475,
                "training_speed": 0.489,
                "lr": [
                    0.0001,
                    0.001
                ]
            }
        }
    }
]


输出样例:
```json
{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "vgg300_ssd_voc",
    "timestamp": 1575045096684,
    "datetime": "2019-11-30 00:31:36",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 14.5297184,
        "training_speed": 0.603,
        "lr": [
            0.0002,
            0.001
        ]
    }
}
```

## 2 评估

In [23]:
cv_stop = k12ai_post_cv_request(
    uri='k12ai/framework/evaluate',
    op='evaluate.stop',
    user='test',
    uuid='voc',
)
k12ai_print(cv_stop)

{
    "req": {
        "op": "evaluate.stop",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "voc",
        "service_params": {}
    },
    "res": {
        "code": 100000,
        "descr": "success"
    },
    "key": "framework/test/voc/evaluate.stop"
}


In [24]:
k12ai_del_data(cv_stop['key'])
k12ai_print(k12ai_get_data(cv_stop['key'], 'error', waitcnt=3))

In [25]:
cv_start = k12ai_post_cv_request(
    uri='k12ai/framework/evaluate',
    op='evaluate.start',
    user='test',
    uuid='voc',
    params=config
)
k12ai_print(cv_start)

{
    "req": {
        "op": "evaluate.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "voc",
        "service_params": {
            "dataset": "default",
            "task": "det",
            "method": "single_shot_detector",
            "data": {
                "num_classes": 21,
                "data_dir": "/data/datasets/cv/VOC07+12_DET",
                "image_tool": "cv2",
                "input_mode": "BGR",
                "keep_difficult": false,
                "workers": 1,
                "mean_value": [
                    104,
                    117,
                    123
                ],
                "normalize": {
                    "div_value": 1,
                    "mean": [
                        104.0,
                        117.0,
                        123.0
                    ],
                    "std": [
                        1.0,
                        1.0,
                        1.0
              

In [26]:
k12ai_del_data(cv_start['key'])

'success'

In [27]:
k12ai_print(k12ai_get_data(cv_start['key'], 'metrics', waitcnt=30))
k12ai_print(k12ai_get_data(cv_start['key'], 'error'))
k12ai_print(k12ai_get_data(cv_start['key'], 'status'))

## 3. 用户自定义模型

In [28]:
net_def = '''plain_net {
    name: "det_7b6a035dfb085fca2dd9bc35939b8add"
     
    layer{
      conv{
        name : "Conv2d_6018"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "x"
        outputs : "Relu_8759"
        layer_params {
          in_channels : "3"
          out_channels : "64"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_8759"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_6018"
        outputs : "Conv2d_9065"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_9065"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_8759"
        outputs : "Relu_3620"
        layer_params {
          in_channels : "64"
          out_channels : "64"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_3620"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_9065"
        outputs : "MaxPool2d_8137"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      pool{
        name : "MaxPool2d_8137"
        layer_builder : "NNTorchLayer"
        layer_mode : MAXPOOL2D
        inputs : "Relu_3620"
        outputs : "Conv2d_1217"
        layer_params {
          kernel_size : "2"
          stride : "2"
          ceil_mode : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_1217"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "MaxPool2d_8137"
        outputs : "Relu_1020"
        layer_params {
          in_channels : "64"
          out_channels : "128"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_1020"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_1217"
        outputs : "Conv2d_4367"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_4367"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_1020"
        outputs : "Relu_4518"
        layer_params {
          in_channels : "128"
          out_channels : "128"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_4518"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_4367"
        outputs : "MaxPool2d_6358"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      pool{
        name : "MaxPool2d_6358"
        layer_builder : "NNTorchLayer"
        layer_mode : MAXPOOL2D
        inputs : "Relu_4518"
        outputs : "Conv2d_3526"
        layer_params {
          kernel_size : "2"
          stride : "2"
          ceil_mode : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_3526"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "MaxPool2d_6358"
        outputs : "Relu_2841"
        layer_params {
          in_channels : "128"
          out_channels : "256"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_2841"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_3526"
        outputs : "Conv2d_4763"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_4763"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_2841"
        outputs : "Relu_2199"
        layer_params {
          in_channels : "256"
          out_channels : "256"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_2199"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_4763"
        outputs : "Conv2d_6272"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_6272"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_2199"
        outputs : "Relu_2306"
        layer_params {
          in_channels : "256"
          out_channels : "256"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_2306"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_6272"
        outputs : "MaxPool2d_3269"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      pool{
        name : "MaxPool2d_3269"
        layer_builder : "NNTorchLayer"
        layer_mode : MAXPOOL2D
        inputs : "Relu_2306"
        outputs : "Conv2d_9436"
        layer_params {
          kernel_size : "2"
          stride : "2"
          ceil_mode : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_9436"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "MaxPool2d_3269"
        outputs : "Relu_3155"
        layer_params {
          in_channels : "256"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_3155"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_9436"
        outputs : "Conv2d_4320"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_4320"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_3155"
        outputs : "Relu_5503"
        layer_params {
          in_channels : "512"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_5503"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_4320"
        outputs : "Conv2d_6001"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_6001"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_5503"
        outputs : "Relu_2697"
        layer_params {
          in_channels : "512"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_2697"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_6001"
        outputs : "MaxPool2d_8588"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      pool{
        name : "MaxPool2d_8588"
        layer_builder : "NNTorchLayer"
        layer_mode : MAXPOOL2D
        inputs : "Relu_2697"
        outputs : "Conv2d_1211"
        layer_params {
          kernel_size : "2"
          stride : "2"
          ceil_mode : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_1211"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "MaxPool2d_8588"
        outputs : "Relu_5533"
        layer_params {
          in_channels : "512"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_5533"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_1211"
        outputs : "Conv2d_8034"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_8034"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_5533"
        outputs : "Relu_5816"
        layer_params {
          in_channels : "512"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_5816"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_8034"
        outputs : "Conv2d_1342"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_1342"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_5816"
        outputs : "Relu_4359"
        layer_params {
          in_channels : "512"
          out_channels : "512"
          kernel_size : "3"
          padding : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_4359"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_1342"
        outputs : "MaxPool2d_6276"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      pool{
        name : "MaxPool2d_6276"
        layer_builder : "NNTorchLayer"
        layer_mode : MAXPOOL2D
        inputs : "Relu_4359"
        outputs : "Conv2d_7495"
        layer_params {
          kernel_size : "3"
          stride : "1"
          padding : "1"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_7495"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "MaxPool2d_6276"
        outputs : "Relu_3916"
        layer_params {
          in_channels : "512"
          out_channels : "1024"
          kernel_size : "3"
          padding : "6"
          dilation : "6"
        }
      }
    }
    layer{
      act{
        name : "Relu_3916"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_7495"
        outputs : "Conv2d_4286"
        layer_params {
          inplace : "True"
        }
      }
    }
    layer{
      conv{
        name : "Conv2d_4286"
        layer_builder : "NNTorchLayer"
        layer_mode : CONV2D
        inputs : "Relu_3916"
        outputs : "Relu_2931"
        layer_params {
          in_channels : "1024"
          out_channels : "1024"
          kernel_size : "1"
        }
      }
    }
    layer{
      act{
        name : "Relu_2931"
        layer_builder : "NNTorchLayer"
        layer_mode : RELU
        inputs : "Conv2d_4286"
        layer_params {
          inplace : "True"
        }
      }
    }
}
'''

In [29]:
config = SSD_VOC_CONFIG
config['solver']['max_epoch'] = 2
config['train']['batch_size'] = 8
config['network']['model_name'] = 'custom_ssd300'
config['network']['net_def'] = net_def

In [33]:
cv_start = k12ai_post_cv_request(
    uri='k12ai/framework/train',
    op='train.start',
    user='test',
    uuid='voc',
    params=config
)
k12ai_print(cv_start)

{
    "req": {
        "op": "train.start",
        "user": "test",
        "service_name": "k12cv",
        "service_uuid": "voc",
        "service_params": {
            "dataset": "default",
            "task": "det",
            "method": "single_shot_detector",
            "data": {
                "num_classes": 21,
                "data_dir": "/data/datasets/cv/VOC07+12_DET",
                "image_tool": "cv2",
                "input_mode": "BGR",
                "keep_difficult": false,
                "workers": 1,
                "mean_value": [
                    104,
                    117,
                    123
                ],
                "normalize": {
                    "div_value": 1,
                    "mean": [
                        104.0,
                        117.0,
                        123.0
                    ],
                    "std": [
                        1.0,
                        1.0,
                        1.0
                 

In [32]:
k12ai_del_data(cv_start['key'])
k12ai_print(k12ai_get_data(cv_start['key'], 'error', waitcnt=3))

In [31]:
k12ai_print(k12ai_get_data(cv_start['key'], 'metrics', waitcnt=30))
k12ai_print(k12ai_get_data(cv_start['key'], 'error'))
k12ai_print(k12ai_get_data(cv_start['key'], 'status'))

[
    {
        "key": "framework/test/voc/train.start/metrics/201912200038",
        "value": {
            "version": "0.1.0",
            "type": "metrics",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "voc",
            "timestamp": 1576773513491,
            "datetime": "20191220003833",
            "metrics": {
                "training_epochs": 1,
                "training_loss": 14.26034546,
                "training_speed": 0.378,
                "lr": [
                    0.0002,
                    0.001
                ]
            }
        }
    }
]
[
    {
        "key": "framework/test/voc/train.start/error/201912200044",
        "value": {
            "version": "0.1.0",
            "type": "error",
            "tag": "framework",
            "op": "train.start",
            "user": "test",
            "service_uuid": "voc",
            "timestamp": 1576773852606,
            "datetime": "201